<a href="https://colab.research.google.com/github/maasanmrs/AIpractice/blob/update/Gemini_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

このプログラムを実行する前に、GeminiのAPIキーを発行してシークレットキーに保管してください
またNDCのドキュメントをドライブに保存してください

NDCのtxtデータはEmbeddingのレポジトリに落ちてます
(余裕があったらRAGのレポジトリにもおいておきますね)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#必要なライブラリのインストール
!pip install langchain
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.

In [3]:
#Google Driveへのアクセス許可
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!pip install -U langchain-community
# install the missing langchain-community package#必要なライブラリインポート
# Embedding用
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
# テキストファイルを読み込む
from langchain.document_loaders import TextLoader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.9 MB/s eta 0:00:00


In [7]:
# 環境変数の準備
import google.generativeai as genai
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [8]:
txt_path = "/content/drive/MyDrive/NDCdata/doc_class.txt"

In [9]:
#テキストデータの読み込み
loader = TextLoader(txt_path)
documents = loader.load()

In [10]:
#テキストデータの分割
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

In [16]:
import pandas as pd
df=pd.DataFrame(data=docs,columns=['id',"source","doc","type"]).drop(columns=["id"])
display(df)

,source,doc,type
0,"(metadata, {'source': '/content/drive/MyDrive/...","(page_content, 国立国会図書館\n「日本十進分類法（NDC）新訂 10 版」\...","(type, Document)"
1,"(metadata, {'source': '/content/drive/MyDrive/...","(page_content, 目次\n本基準の凡例 .......................","(type, Document)"
2,"(metadata, {'source': '/content/drive/MyDrive/...","(page_content, 本基準の凡例\n\n\n各項目には、分類基準に関連する分類記...","(type, Document)"
3,"(metadata, {'source': '/content/drive/MyDrive/...","(page_content, \n\n固有補助表を使用しない場合は、個別に明記する。\n\...","(type, Document)"
4,"(metadata, {'source': '/content/drive/MyDrive/...","(page_content, 情報産業．情報サービス\n\n個々の企業史・誌も，ここに収める...","(type, Document)"
...,...,...,...
215,"(metadata, {'source': '/content/drive/MyDrive/...","(page_content, 団体，企業体．会社誌，研究調査機関，教育・養成機関\n\nこれ...","(type, Document)"
216,"(metadata, {'source': '/content/drive/MyDrive/...","(page_content, 分を使用し，地理区分を行わない；ただし，項目名が団体に該当する...","(type, Document)"
217,"(metadata, {'source': '/content/drive/MyDrive/...","(page_content, 115\n\n －088 資料集\n特定主題についての史料集に...","(type, Document)"
218,"(metadata, {'source': '/content/drive/MyDrive/...","(page_content, 相関索引・使用法編\n使用法\nI 2.1 4）複数主題\nも...","(type, Document)"


In [17]:
#分割したドキュメントをEmbedding
docs=list(docs)
result = genai.embed_content(
    model="models/embedding-001",
    content=df["doc"],
    task_type="retrieval_document",
    title="Embedding of list of strings")
df["Embedding"]=result["embedding"]

In [18]:
#得られたベクトルを標準化
import numpy as np
def normalize_vector(v):
    norm = np.linalg.norm(v)
    if norm == 0:
       return v
    return v / norm

df["Embedding"]=df["Embedding"].map(normalize_vector)
passage_embeddings=df["Embedding"]

In [19]:
#queryをEmbedding
result = genai.embed_content(
    model="models/embedding-001",
    content="""
    手宮線は、全国で３番目、北海道では最初に開通した路線です。
    南小樽～手宮間は、わずか２．８ｋｍの短い路線ですが、北海道開拓の物資輸送の拠点として発展しました。
    しかし戦後、利用客は減少を続け、昭和６０年１１月、手宮線は廃止されました。
    北海道鉄道発祥の地である手宮駅の跡地は、現在、小樽市総合博物館として活用され現存する日本最古の機関庫とともに、貴重な車両が保存されています。
    """,
    task_type="retrieval_document",
    title="Embedding of list of strings")
query_embedding = normalize_vector(result["embedding"])

In [20]:
#得られたベクトルの内積を計算
scores = []
for content in passage_embeddings:
  scores.append(np.dot(content,query_embedding))

In [26]:
#内積を降順に並べ替えた時のインデックスを取得
def get_sorted_indices(lst):
    return [i for i, v in sorted(enumerate(lst), key=lambda x: x[1],reverse = True)]
scores_indices = get_sorted_indices(scores)
print(np.argsort([3,5,7,3,2])[::-1]) ＃こちらでも同様
print(get_sorted_indices([3,5,7,3,2]))

[2 1 3 0 4]
[2, 1, 0, 3, 4]


In [27]:
#コサイン類似度の高い３つの文章を出力
print(df["doc"][scores_indices[0:3]])

38    (page_content, 17\n\n（2）個々の戦争における戦没者・従軍者の列伝は，...
42    (page_content, 19\n\n（3）土地，風土のことよりも社会事情（政治・経済...
69    (page_content, 35\n\n細目表に掲出されていなくても，独自の学説・体系を...
Name: doc, dtype: object


次に実際にLLMを通して効果を確認してみます

今回使うのはLlama2を日本語に関して教科学習させた
"elyza/ELYZA-japanese-Llama-2-7b-instruct"

In [28]:
# パッケージのインストール
!pip install transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 4.5 MB/s eta 0:00:00


In [31]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# トークナイザーとモデルの準備
tokenizer = AutoTokenizer.from_pretrained(
    "elyza/ELYZA-japanese-Llama-2-7b-instruct"
)
model = AutoModelForCausalLM.from_pretrained(
    "elyza/ELYZA-japanese-Llama-2-7b-instruct",
    torch_dtype=torch.float16,
    device_map="auto",
    offload_folder="offload"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

In [32]:
#プロンプトを作成する
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """
あなたは誠実で優秀な日本人のアシスタントです。
与えられた文章に対して適切な分類項目を答えてください
この際分類項目に関する情報が与えられます
"""

def q(text):
	prompt = "{bos_token}{b_inst} {system}{prompt} {e_inst} ".format(
	    bos_token=tokenizer.bos_token,
	    b_inst=B_INST,
	    system=f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}",
	    prompt=text,
	    e_inst=E_INST,
	)
	with torch.no_grad():
	    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
	    output_ids = model.generate(
	        token_ids.to(model.device),
	        max_new_tokens=1024,
	        pad_token_id=tokenizer.pad_token_id,
	        eos_token_id=tokenizer.eos_token_id,
	    )
	output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)
	print(output)

In [34]:
#RAGする前の出力
text = """手宮線は、全国で３番目、北海道では最初に開通した路線です。
    南小樽～手宮間は、わずか２．８ｋｍの短い路線ですが、北海道開拓の物資輸送の拠点として発展しました。
    しかし戦後、利用客は減少を続け、昭和６０年１１月、手宮線は廃止されました。
    北海道鉄道発祥の地である手宮駅の跡地は、現在、小樽市総合博物館として活用され現存する日本最古の機関庫とともに、貴重な車両が保存されています。
    """
q(text)

KeyboardInterrupt: 

In [ ]:
#RAGしてきた文章を元のプロンプトに加える
query = text +str(df["doc"][scores_indices[0]])
print(query)

In [ ]:
q(query)